## AI agenter

![img_1.png](images/img_1.png)

In [ ]:
!curl https://www.klarna.com/us/shopping/public/openai/v0/api-docs/ -o openapi.yaml

# You can try to replace this with your own openapi spec

In [ ]:
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import yaml

with open("openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
klarna_api_spec = reduce_openapi_spec(raw_openai_api_spec)

print(klarna_api_spec)

In [ ]:
import os
import dotenv

dotenv.load_dotenv()

from langchain_openai import AzureChatOpenAI

if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    raise ValueError("Please set the environment variable AZURE_OPENAI_ENDPOINT")

if not os.getenv("AZURE_OPENAI_API_KEY"):
    raise ValueError("Please set the environment variable AZURE_OPENAI_API_KEY")

if not os.getenv("AZURE_CHAT_DEPLOYMENT"):
    raise ValueError("Please set the environment variable AZURE_CHAT_DEPLOYMENT")

model = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_CHAT_DEPLOYMENT"],
    api_version="2023-03-15-preview"
)

In [ ]:
from langchain_community.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper


api_agent = planner.create_openapi_agent(
    api_spec=klarna_api_spec,
    requests_wrapper=RequestsWrapper(), # headers = {"Authorization": f"Bearer {Your API KEY}"}
    llm=model,
    allow_dangerous_requests=True, # Trust me, nothing suspicious here
    agent_executor_kwargs={"handle_parsing_errors": True}
)

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm_template =""" Your role is to be a personal shopping assistant
{
    "task": "helping a user find clothes based on {question}",
    "items": {context}
}
"""

agent_chain = (
        RunnableParallel({"context": api_agent, "question": RunnablePassthrough()})
        | ChatPromptTemplate.from_template(llm_template)
        | model
        | StrOutputParser()
)


In [ ]:
print(agent_chain.invoke("It is hot out, i need some linen clothing for men in size medium, can you give me some options?"))